In [1]:
using Laplacians

objc[44611]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[44611]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[44611]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[44611]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/se_6/"

"../graphs/se_6/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
x = readFromFile(graphp * "x.vec");

In [8]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [10]:
@time F = treeSolver(tree)
@time F(b);

  0.551897 seconds (1.25 M allocations: 86.056 MB, 3.56% gc time)
  0.084836 seconds (2.00 M allocations: 38.109 MB, 11.49% gc time)


In [12]:
# a run on standard Float64 data types
numIts = 200;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 275248.0490339541
Working on iteration 20 with a-norm error 37221.002833847386
Working on iteration 30 with a-norm error 2872.0353664786153
Working on iteration 40 with a-norm error 720.9238241848179
Working on iteration 50 with a-norm error 111.41249096691962
Working on iteration 60 with a-norm error 5.757656552309614
Working on iteration 70 with a-norm error 0.9712163561382174
Working on iteration 80 with a-norm error 0.25193610214413953
Working on iteration 90 with a-norm error 0.01889657592714628
Working on iteration 100 with a-norm error 0.005032927009409369
Working on iteration 110 with a-norm error 0.0007184359381484255
Working on iteration 120 with a-norm error 8.893801479762932e-5
Working on iteration 130 with a-norm error 1.1743933280725218e-5
Working on iteration 140 with a-norm error 9.698601822599699e-6
Working on iteration 150 with a-norm error 9.155043364065166e-6
Working on iteration 160 with a-norm error 9.245591999040605e-6
Wo

In [13]:
numIts = 200
for hp in [64,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 275248.04894769826
Working on iteration 20 with a-norm error 24904.73149944613
Working on iteration 30 with a-norm error 2871.337849309644
Working on iteration 40 with a-norm error 509.725659815555
Working on iteration 50 with a-norm error 53.97540191636309
Working on iteration 60 with a-norm error 3.5937126926558944
Working on iteration 70 with a-norm error 0.31080315533400915
Working on iteration 80 with a-norm error 0.06932782210719368
Working on iteration 90 with a-norm error 0.015099771218466776
Working on iteration 100 with a-norm error 0.001338055132368249
Working on iteration 110 with a-norm error 9.89309717422729e-5
Working on iteration 120 with a-norm error 2.353583212071341e-5
Working on iteration 130 with a-norm error 1.6881595218232233e-6
Working on iteration 140 with a-norm error 4.2897719074003256e-7
Working on iteration 150 with a-norm error 1.400190569181171e-7
Working on iteration 160 with a-norm error 1.2761273922762324e-7
Wo